In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pyproj import Transformer
from tqdm import tqdm

In [3]:
Town_data = gpd.read_file("./Town/TOWN_MOI_1120825.shp")
Big_Taipei = Town_data[(Town_data['COUNTYNAME'] == '臺北市') | (Town_data['COUNTYNAME'] == '新北市')]


transformer = Transformer.from_crs("EPSG:3826", Big_Taipei.crs, always_xy=True)


def get_district(x, y):
    point = Point(x, y)
    point_gdf = gpd.GeoDataFrame([{'geometry': point}], crs=Big_Taipei.crs)
    point_in_district = gpd.sjoin(point_gdf, Big_Taipei, how="inner", predicate='within')
    if not point_in_district.empty:
        return point_in_district['TOWNNAME'].iloc[0]
    return None


df = pd.read_csv('points_taipei.csv')

df['district'] = None


for index, row in tqdm(df.iterrows(), total=df.shape[0]):

    x_min, y_min = transformer.transform(row['x1'], row['y1'])
    x_max, y_max = transformer.transform(row['x2'], row['y2'])
    

    districts = set()
    

    for x, y in [(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)]:
        district = get_district(x, y)
        if district:
            districts.add(district)    

    df.at[index, 'district'] = list(districts)

print(df)
df.to_csv('updated_points_taipei.csv', index=False)


 13%|█████████▊                                                                   | 3527/27661 [04:21<29:46, 13.51it/s]


KeyboardInterrupt: 

In [4]:
Town_data = gpd.read_file("./Town/TOWN_MOI_1120825.shp")
Big_Taipei = Town_data[(Town_data['COUNTYNAME'] == '臺北市') | (Town_data['COUNTYNAME'] == '新北市')]


transformer = Transformer.from_crs("EPSG:3826", Big_Taipei.crs, always_xy=True)


def get_district(x, y):
    point = Point(x, y)
    point_gdf = gpd.GeoDataFrame([{'geometry': point}], crs=Big_Taipei.crs)
    point_in_district = gpd.sjoin(point_gdf, Big_Taipei, how="inner", predicate='within')
    if not point_in_district.empty:
        return point_in_district['TOWNNAME'].iloc[0]
    return None


df = pd.read_csv('points_new_taipei.csv')

df['district'] = None


for index, row in tqdm(df.iterrows(), total=df.shape[0]):

    x_min, y_min = transformer.transform(row['x1'], row['y1'])
    x_max, y_max = transformer.transform(row['x2'], row['y2'])
    

    districts = set()
    

    for x, y in [(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)]:
        district = get_district(x, y)
        if district:
            districts.add(district)    

    df.at[index, 'district'] = list(districts)

print(df)
df.to_csv('updated_points_new_taipei.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████| 209748/209748 [4:22:12<00:00, 13.33it/s]


            x1       y1      x2       y2 district
0       278508  2777870  278608  2777970    [林口區]
1       278508  2777970  278608  2778070    [林口區]
2       278508  2778070  278608  2778170    [林口區]
3       278508  2778170  278608  2778270    [林口區]
4       278508  2778370  278608  2778470       []
...        ...      ...     ...      ...      ...
209743  351508  2767770  351608  2767870       []
209744  351608  2766970  351708  2767070    [貢寮區]
209745  351608  2767070  351708  2767170    [貢寮區]
209746  351608  2767270  351708  2767370    [貢寮區]
209747  351608  2767370  351708  2767470    [貢寮區]

[209748 rows x 5 columns]
